# Fundamentals of Script Writing in Psychopy

Psychopy is a unique and easy-to-use resource for writing experimental psychological scripts. It can be used to run infinite interactive paradigms using various forms of stimulus presentation. This notebook walks you through putting together the bare-bones of a psychopy script, which you can then personalize to your hearts desire. Let's get started!

*Please note that this notebook walks you through CODER VIEW in psychopy. This does not apply to builder view. Well, it might. I've never actually used builder view. But I would just stick with coder view to practice python skills!

### Getting Started

The first thing you want to do in your script is import all the python packages you will need for your experiment to run correctly. These include everything from a package to randomize lists to the package that allows you to show visual stimuli. Below are the bare basics to import. 

In [ ]:
import os 
import sys
import numpy as np #extension of the python programming language, adds support for experimental data
import psychopy.gui #fetches default qui handler
import psychopy.visual #fetches extension that supports handling visual stimuli
import psychopy.event #package offers easy way to collect responses--recording key presses, etc.
import psychopy.core #fetches basic functions, including timing
from random import randint #function allows randomization of intervals
from random import shuffle #function allows you to shuffle lists in one command

These are the basics that I always start with. However, more complex experiments often use paradigms that require extra functions that are not covered by the above packages. You can always look up extra functions and available packages online! At the end of this tutorial, I will write a few extra that often come in handy, and what they can do for you.

### Subject ID

When you start your script for a participant, you are going to want a way to mark down their subject ID. The following code will cause a box to pop up when you first run the script where you can input that information. 

In [ ]:
gui = psychopy.gui.Dlg()
gui.addField("Subject ID:")
gui.show()
subj_id = gui.data[0]

### Creating a file path for response data

In [ ]:
data_path = "ExperimentX"+str(subj_id)+".csv"

You need to create a path for the data you collect. The above code creates a new .csv (comma separated value) file that the data will now be funnelled into. I always use .csv files as they can be read easily by excel, as well as other texteditors. 

Put the name of your experiment where *ExperimentX* is written. Often I will also deliniate what type of data I am collecting as well in this area. So if my experiment is called Fear Faces, and I am collecting rating data, I would write "FF_Responses". It is always better to label as clear as possible to prevent future confusion.

Note that the middle of the code links to the subj_id we collected above. This will insert the subject ID you enter into the file name of the data_path. So if I was running my Fear Faces experiment, and I was running particpant 05, the data file on my computer would look like: FF_Responses05.csv

In [ ]:
while os.path.exists(data_path): 
    print "CHECK SUBJECT NUMBER"
    subj_id = subj_id+"000"
    data_path = "ExperimentX"+str(subj_id)+".csv"

Sometimes you make mistakes and accidentally enter the wrong subject ID number when running a particpant. This could cause the program to overwrite previous collected data (if you use a subject ID that has already been used). This code is a failsafe against that happening. It does two things.

First, it changes the subj_id from whatever you input by adding 000 in front of it. This will prevent files from being overwritten.

Second, it will print in the shell (which you can check after running the experiment) "CHECK SUBJECT NUMBER", which will alert you to the mistake. 

While this code is not necessary, I always put it in, just in case.

### Declaring Experimental Variables 

This is very dependent on the type of experimental parameters you are working with. For this tutorial, we will be using the Fear Face example, with a block design. This study includes looking at a block of neurtal faces, collecting rating data, and then looking at a block of fearful faces and collecting rating data. 

In [ ]:
cl = psychopy.core.Clock();  

This declares a clock for tracking run time, reaction time, or whatever you want to time within your experiment. You can delcare multiple clocks if you are interested in timing mulitple aspects (i.e., a clock for time-stamping reaction time, and a clock to mark onsets of image stimuli). Just make sure you name the clocks differently.

In [ ]:
n_trials=30; #how many unique images/stimuli there are TOTAL
n_reps=1; #how many times to repeat through the entire stim-set per condition
n_vars=5; #Subject ID, Stim ID, Trail #, Subj Response, Reaction Time


In the variables section, this is where you define out your experiment parameters. So for Fearful Faces, I have 2 conditions, neutral and fearful, with 15 images in each. However, I only need to define how many trials per each condition, as you will see later. I am going to run through each only once, to keep things simple.

In [ ]:
Neutral_stim_list = ["image.jpg", "image2.jpg"];
Fear_stim_list = ["otherimage.jpg", "otherimage2.jpg"];
shuffle(Neutral_stim_list);
shuffle(Fear_stim_list);

In the stim lists, you are listing out the actual names of your stimuli files, EXACTLY as they are named/saved on your computer. The script has to march exactly the files so the program knows what to pull. By documenting the label of each stimuli, you can later do item analysis. We will later define where on the computer to pull them from. This is also where I use the shuffle function we imported at the beginning. I want my stim to be presented in a random order for each participant. This function ensures that every time I run the script, it will re-shuffle my list of stimuli. 

In [ ]:
X=np.zeros((n_trials,n_vars));

Now we need to create a blank vector to contain the data we will be collecting. For my experiment, I will have 30 trials TOTAL, so I need a vector that has 30 rows (it doesn't include the header, so you don't need to factor that in), and 5 columns for my 5 variables.

### Creating Your Experiment Window

In [ ]:
win = psychopy.visual.Window(
    size=[1280, 800],
    units="pix",
    fullscr=False,
    color=[-1, -1, -1]
)

You need to create the "window" where your experiment will take place. This is what the participant will see.

Size depends on your computer screen dimensions. I use a macbook pro 13', and these dimensions work great for me.

Units changes depending on your stimulus form. Since I am using jpg images, I want pixels. If you are using something else, look on the psychopy website to see what you should change it to. 

fullscr denotes whether or not it will be fullscreen when you run the experiment. I like to keep this False while I am working, so I can get in and out of the script easier. Just make sure that you change it to True when you start running it for participants.

color is what color you want the background to be. I use black. 
[1,1,1] is white
[0,0,0] is grey
[-1,-1,-1] is black
[1.0,-1,-1] is red
[1.0,0.6,0.6] is pink

This should be all you need for a basic background window. However, there are other parameters you can specify. You can find these on psychopy.org. This window will not change throughout your experiment. It is the meta-space that all other screens are being projected onto.

### Defining Text / Instruction Screens

This where you define the text or intructions you want the participant to see throughout the experiment. It is best to define it all in one place so it doesn't distract or muddle your code later on. It makes for a cleaner script. For my experiment, I want a welcome screen, an instruction screen for both neutral and fearful, a rating screen where they will respond, and a goodbye screen. This is also where I will create the fixation screen. 

In [ ]:
welcome_text="Welcome! Press any key to continue...";
text = psychopy.visual.TextStim(
    win=win,
    text=welcome_text,
    color=[1, 1, 1]
)

Instr_text = "You will see a series of images on the screen. After each image, you will be asked to rate how positive or negative you think each image is on a scale of 1-7. Please answer as quickly and honestly as you can. Press any key to begin the experiment...";
text = psychopy.visual.TextStim(
    win=win,
    text= Neutral_instr_text,
    wrapWidth=1000,
    color=[1, 1, 1]
)

Rating_text = "How positive or negative is the image?\n\n\n\n\n\nvery negative......................neutral.....................very positive\n1         2         3         4        5        6        7";
text = psychopy.visual.TextStim(
    win=win,
    text=Rating_text,
    wrapWidth=1000,
    color=[1, 1, 1]
)

Goodbye_text= "The experiment is now finished. Please let the experimentor know."
text = psychopy.visual.TextStim(
    win=win,
    text=Goodbye_text,
    wrapWidth=1000,
    color=[1, 1, 1]
)

fixation = "+";
text = psychopy.visual.TextStim(
    win=win,
    text=fixation,
    color=[1, 1, 1]
)


Now I have all my non-stim screens defined. I just need to call them later in my script in order for them to show up. 

A few things to note:
You will notice that on the longer texts, I defined "wrapWidth". Psychopy automatically centers and condenses the script within a set area on the screen. It is only noticable with longer text (i.e., I didn't need it when defining the fixation). Setting the WrapWidth to 1000 keeps it from hitting the edges of the computer screen, but also prevents it from being condensed into the middle of the page. You may also want to change text size, which is just denoted by 'height = #;'. A good place to start is 35. It is hard to tell what you will want until you run the script and get a feel for how things look.

You will also notice that on the rating text, I explicitly wrote (or "drew") it out. The \n is the equivalent of a paragraph break. This is also something you will have to fiddle with after running the script once or twice. You may need fewer or more spaces to make the numbers and descriptive titles (very negative, etc) match up. You may also want more or less paragraph breaks in between them. 


### On to the meat of things...

Now that we have all our variables, text, and window defined, we can start coding the actual experiment itself. 